In [33]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Wyświetla wszystkie wiersze
pd.set_option('display.max_columns', None)  # Wyświetla wszystkie kolumny
pd.set_option('display.width', 1000)  # Szerszy output, by uniknąć zawijania
pd.options.display.float_format = '{:.2f}'.format

import sys
import os
sys.path.append(os.path.abspath(".."))

from bot.data.historic.binance_client_wrapper import BinanceClientWrapper
from bot.data.historic.binance_data_fetcher import BinanceDataFetcher
from bot.core.utils import Utils

# client = BinanceClientWrapper()
# client.get_klines('BTCUSDC', '1h', 1735689600000, 1740614400000)

fetcher = BinanceDataFetcher()
df = fetcher.fetch_klines_dates('BTCUSDT', '5m', '2025-01-01 00:00:00', '2025-01-31 23:59:59', 0) # "%Y-%m-%d %H:%M:%S"
df = Utils.add_stoch_rsi(df)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
print(df);

Liczba zapytań do API: 9
                timestamp      open      high       low     close  volume  is_closed   symbol interval  stoch_rsi_k  stoch_rsi_d
0     2025-01-01 00:00:00  93576.00  93702.15  93537.50  93661.20   45.94       True  BTCUSDT       5m          NaN          NaN
1     2025-01-01 00:05:00  93661.20  93678.02  93600.00  93607.10   71.95       True  BTCUSDT       5m          NaN          NaN
2     2025-01-01 00:10:00  93607.10  93656.19  93489.03  93656.18   57.97       True  BTCUSDT       5m          NaN          NaN
3     2025-01-01 00:15:00  93656.19  93840.05  93614.95  93796.35   39.74       True  BTCUSDT       5m          NaN          NaN
4     2025-01-01 00:20:00  93796.35  93898.05  93707.81  93740.00   38.38       True  BTCUSDT       5m          NaN          NaN
5     2025-01-01 00:25:00  93740.00  93761.90  93721.00  93761.90   17.29       True  BTCUSDT       5m          NaN          NaN
6     2025-01-01 00:30:00  93761.90  93963.67  93761.89  93882.92   41.2

In [36]:
import pandas as pd

def find_stoch_rsi_periods(df, threshold=20, min_length=15):
    """
    Znajduje okresy, gdzie stoch_rsi_k pozostaje poniżej threshold przez min_length świec
    i zwraca listę DataFrame'ów obejmujących te okresy.
    
    :param df: DataFrame zawierający kolumnę 'stoch_rsi_k' i 'timestamp'.
    :param threshold: Próg dla stoch_rsi_k (domyślnie 20).
    :param min_length: Minimalna liczba świec poniżej threshold (domyślnie 20).
    :return: Lista DataFrame'ów spełniających warunek.
    """
    below_threshold = df['stoch_rsi_k'] < threshold
    periods = []
    start_idx = None

    for i in range(len(df)):
        if below_threshold.iloc[i]:
            if start_idx is None:
                start_idx = i  # Początek serii świec poniżej threshold
        else:
            if start_idx is not None and (i - start_idx) >= min_length:
                # Tworzymy DataFrame dla tego okresu i dodajemy do listy
                periods.append(df.iloc[start_idx:i].copy())
            start_idx = None

    return periods

# Wywołanie funkcji na DataFrame 'df'
stoch_rsi_dfs = find_stoch_rsi_periods(df)

# Wyświetlenie wyników
for idx, df_period in enumerate(stoch_rsi_dfs):
    print(f"Okres {idx + 1}:")
    print(df_period)
    print("-" * 40)


Okres 1:
              timestamp     open     high      low    close  volume  is_closed   symbol interval  stoch_rsi_k  stoch_rsi_d
41  2025-01-01 03:25:00 93830.89 93848.98 93830.88 93830.89    6.95       True  BTCUSDT       5m         3.96        22.64
42  2025-01-01 03:30:00 93830.90 93848.00 93828.01 93829.13   17.10       True  BTCUSDT       5m         1.24         9.20
43  2025-01-01 03:35:00 93829.13 93850.31 93800.00 93850.31   13.38       True  BTCUSDT       5m         3.63         2.94
44  2025-01-01 03:40:00 93850.31 93891.15 93840.00 93857.04   37.35       True  BTCUSDT       5m         7.01         3.96
45  2025-01-01 03:45:00 93857.03 93867.35 93728.22 93786.19   25.42       True  BTCUSDT       5m         7.01         5.89
46  2025-01-01 03:50:00 93786.18 93812.00 93786.18 93801.66    8.28       True  BTCUSDT       5m         6.15         6.73
47  2025-01-01 03:55:00 93801.66 93838.04 93801.65 93838.04    6.97       True  BTCUSDT       5m         9.34         7.50
48  202